# Third Hands-On Session
---
- Implementing and studying the properties of a FODO cell/lattice
- Computing closed optics solutions
- The concept of Tune and FODO cell phase advance
- Introducing Sector Bends
---

#### Setup of your Python kernel
Let's load our standard library functions, and setup matplotlib for nice inline plots

In [ ]:
# Import custom tracking functions and useful libraries
from tracking_library import *

# Set up plotting
params = {
    "legend.fontsize": "x-large",
    "figure.figsize": (15, 5),
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
plt.rcParams.update(params)

### Exercise 15

Let's start working with FODO cells. This is one of the most fundamental lattice topologies. As in [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/), we define $L$ as the length of one drift, i.e. the total length of the cell ($L_{FODO}$) is $2L$. 

<p align="center">
<img src=./_img_exercises/slide_fodo.png width="60%">
</p>

1. Define a **FODO beamline** that **starts and ends in the middle of a focusing quadrupole**. 
    
    For the moment, we can define arbitrarily the $f$ and $L$ parameters: we chose 2.5 m and 1 m, respectively. 
    
2. Track a particle that has an initial offset $x = 1$ mm and no angle $x' = -0.1$ mrad, and plot its position $x$ and angle $x'$ along the beam line.

**Hint 1:** A half-quadrupole has double the focal length than a full quadrupole.

**Hint 2:** To better visualize the trajectory of the particle(s), always split the drifts in several shorter drifts (e.g. $10$ drifts, each $L/10$ long).

**Hint 3:** After having plotted $x$ trajectory on a plot, one can create a second vertical axis that shares the same horizontal axis with `plt.twinx()`

#### SOLUTION 
The first part of the exercise is to define a FODO cell structure.  In its simpler version, it consists on two thin quadrupoles of opposite focal length, f, spaced by two drifts of length L (2*L is the total length of the cell). 

We could start the FODO from an arbitrary point inside it, but we decided to start describing it from the center of a focusing quadrupole.

We also suggest to split the drift in small drifts, say 0.1 m long each.

In [ ]:
# build your beamline here...

We can now proceed to generate a particle and transport it along the beamline:

In [ ]:
# track and plot the particle trajectory here...

### Exercise 16

Plot the position $x$ and angle $x'$ through $N=100$ equal FODO cells, play with different values of the focal length $f$ and explore whether you can make the oscillations grow.

**Hint:** Look back at [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/):

<p align="center">
<img src=./_img_exercises/slide_stability.png width="60%">
</p>

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 17

1. Transport a sigma matrix (e.g.$\beta$ = 3 [m], $\gamma$ = 0.5 [1/m], $\epsilon$ = 1) through a beamline made of 8 FODO cells (with, for example, $f=2.5$ and $L=1$ as before), and plot the evolution of the sigma matrix elements ($\sigma_{11}$, $\sigma_{22}$, $\sigma_{12}$) along the beamline. Do you see a periodic structure?

2. Try to vary the initial sigma matrix parameter by **try and error** that reproduces itself at the end of the beam line (8 FODO cells). You will quickly realise that this is extremely difficult if not impossible. 

**Hint 1**: (optional) you might want to use an **interactive plot** - see in the previous exercises how to make one

**Hint 2**: **try!**, but **don't waist to much time with try and error** approaches (that's not your job as accelerator physicist!), and **move on** to the next exercise...

### SOLUTION 1

In [ ]:
# code here your solution...

### SOLUTION 2

In [ ]:
# code here your solution...

---
### Important observation

Remember that we introduced the sigma matrix with the following equation:

\begin{equation}
\Sigma = 
    \left[
    \begin{array}{cc}
    \sigma_{xx}  & \sigma_{xx'}\\
    \sigma_{x'x} & \sigma_{x'x'}
    \end{array}
    \right] =
    \epsilon
    \left[ 
    \begin{array}{cc}
        \beta   & -\alpha\\
        -\alpha & \gamma
    \end{array}
    \right] 
\end{equation}

Note that, in the case one sets $\epsilon=1$, the element $\sigma_{xx}$ (or `sigma11` in the notation of our python toolbox) is equal to $\beta$!
=> This means that we developed a simple methode to plot the $\beta$ function along a beamline (provided a given initial condition).

---

### Exercise 18

To simplify the problem, **recall** [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/):

<p align="center">
<img src=./_img_exercises/slide_fodo_periodic.png width="60%">
</p>

In a FODO cell the $\beta$ function reaches a maximum (or a minimum) in the middle of the quadrupoles, i.e. the $\alpha$ function is zero.
If one models the FODO cell starting from the middle of a quadrupole (as we do), then at least one parameter is fixed!

Even better: one can use the equation form [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/) to find the theoretical solution.

#### SOLUTION 1

In [ ]:
# code here your solution...

#### SOLUTION 2

We can use the learned during [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/) for this particular configuration:

In [ ]:
# code here your solution...

## General Solution (NOT limited to FODO cells!)

**Recall** [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/). In a periodic beam line, the transfer matrix assumes the form:

\begin{equation}
M = 
    \left[
    \begin{array}{cc}
    \cos(\mu) + \alpha_0 \sin(\mu)  & \beta_0 \sin(\mu)\\
    - \gamma_0 \sin(\mu) & \cos(\mu) - \alpha_0 \sin(\mu)
    \end{array}
    \right]
\end{equation}

where $\alpha_0$, $\beta_0$, $\gamma_0$ are the initial (and final) **Twiss values** and $\mu$ is the phase advance (or **tune**, if we talk about a whole ring) of the associated beamline.
From this expression, one can extract the periodic solution of any beamline, assuming it exists!

**Note:** Since the determinant of $M$ must be one, then one can verify that the trace of the matrix R must be $|\mathrm{Tr}(M)| \le 2$ to ensure the existence of such a solution!

Those concepts are implemented in the function `twiss()` from our toolbox:

In [ ]:
from tracking_library import twiss

help(twiss)

**Note:** We can apply this function to any beamline, e.g. a series of quadrupoles and drifts, e.g.:

In [ ]:
# create a random beamline of many elements
beamline = 5 * (10 * D(0.5 / 10) + Q(1) + 10 * D(2 / 10) + Q(-0.8) + 10 * D(1.5 / 10) + Q(0.3) + 10 * D(1.5 / 10))

# compute the closed solution
(tune, beta, alpha, gamma) = twiss(beamline)

# assemble the sigma matrix
sigma_matrix = np.array([[beta, -alpha], [-alpha, gamma]])

# print out the values
print(f"The total phase advance (or tune) of the beamline is {tune}.")
print(f"The sigma matrix is:\n {sigma_matrix}")
print(f"which has determinant equal to: {np.linalg.det(sigma_matrix):.3f}.")

- **Can you comment why the determinant of the obtained sigma matrix is equal to 1?**

### Exercise 19

Consider again a FODO cell, and choose $f$ and $L$ making sure it allows for having a periodic solution (i.e. $f > L_{FODO}/4$ - see again [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/)). 
This time, **model the FODO cell starting from the middle of a drift**, i.e. `D(L/2) + Q(f) + D + Q(-f) + D(L/2)`.

1. Compute the periodic $\Sigma_0$ matrix using the Twiss parameters obtain for the newly defined `twiss` function (assuming $\epsilon=1$).
2. Transport the periodic $\Sigma$ matrix along the FODO and convince yourself that the $\Sigma_s$ matrix at the end of the FODO is indeed equal to the one at the start, $\Sigma_0$.

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 20
Write down the numerical values of initial beam matrix $\Sigma_0$, then build a beam line made of $15$ consecutive cells by changing the definition of the lattice and then, using $\Sigma_0$ with the noted-down numbers, prepare a plot of the beam sizes along the $15$ cells. 
Is this also periodic?

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 21

Consider the previous FODO cell starting in the middle of the drift, and stable. 
Consider a single particle with some non-zero initial $x_0$ and $x_0'$ (e.g. $x_0 =.24$; $x' = 2.3$), and plot the $x$, $x'$ coordinates at the end of each FODO cell for a beamline made of 100 cells. 

**Hint:** You should use both `getEquivalentElement` to obtain the single FODO cell element, and `transportParticles` function to track the particle over several of such a cell.

**Hint:** use `plt.scatter(output['x'], output['px'])` to plot all phase-space coordinates.

#### SOLUTIONS
For plotting the trace space evolution of a particle we need to observe it turn-after-turn. The natural way to do that is to compress the beam line (in general composed by more than one element) in a single transformation: the one-turn-matrix, which can be computed using the `getEquivalentElement` function previously introduced.

In [ ]:
# code here your solution...

### Exercise 22
Consider again the previous FODO cell, and a particle starting with non-zero coordinates. Plot the position of the particle vs the number of turns. 

1. What do you observe?
2. Can you infer the tune? 
3. How many turns (or FODO cells) are needed for the particle to complete one full oscillation?
4. What changes if you start from a different initial coordinate?

#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 23

Consider a single FODO cell of total length $L_{FODO}=2$ m. 

1. Can you find $f$ such that the FODO cell phase advance is $\mu = \pi/2$, i.e. 90 deg phase-advance?
2. Repeat this cell 100 times, and plot the trajectory, in phase space, of a particle with x=1 mm and xp=0 cell after cell: what do you observe?
3. (Optional) Show that the similar results are obtained irrespectively if you start your FODO from the middle of a quadrupole or of a drift.

**Hint:** Look back at [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/):

<p align="center">
<img src=./_img_exercises/slide_fodo_phase.png width="60%">
</p>

and recall that $\mathrm{Tr}(M) = 2 \cos(\mu)$. 
With some math, one can prove that:

\begin{equation}
\sin(\mu/2) = \frac{L_{FODO}}{4f}
\end{equation}



#### SOLUTION

In [ ]:
# code here your solution...

### Exercise 24

Build a beam line made of 6 FODO cells with $L_{cell} = 4$ m and **Adjust the value of f** such that one cell phase advance is **60 degrees**.

1. Compute the closed optics using the `twiss()` function on a **single cell**, and plot the **beta function** along the whole beamline. Can you also compute the closed optics solution **applying twiss() to the whole beamline**? Does the solution differ? Why? What is the "tune" of the whole beamline?
2. Introduce **sector bends** ($L_{bend} = L_{DRIFT}/2 = 1$ m, and bending angle $\phi=10$ degrees) in the middle of all drifts of the FODO cells, without changing the total length of the FODO cell. How does the optics change?
3. Try to adjust the focal length of the quadrupoles to recover **60 degrees** phase advance **per cell**.

**Hint:** Our toolbox library implements also sector bends: `B(phi, L)` defined by bending angle (`phi`) and bend lenght (`L`).

In [ ]:
# Import the sector bend model
from tracking_library import B

help(B)

### SOLUTION 1

In [ ]:
# code here your solution...

### SOLUTION 2

In [ ]:
# code here your solution...

### SOLUTION 3

In [ ]:
# code here your solution...

## Well done !!

=> **If you still have time, continue your learning with the following [notebook](./04_Dispersion.ipynb)**...
